In [188]:
import numpy as np
import tensorflow as tf
import json
import random
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [189]:
with open('First Thousand Jan.json', 'r') as f:   
    sequences = json.load(f)
    sequences = sequences['Sequences']

In [190]:
s_sequences = [s['S Unit Sequence'] for s in sequences if s['S Unit Sequence'] is not None]
max_length = 0
min_length = 1000000
for s in s_sequences:
    max_length = max(max_length, len(s))
    min_length = min(min_length, len(s))
print(max_length)
print(min_length)

3831
3801


In [428]:
# Set seed for random so it's repeatable
random.seed(111)
rand_length = random.randint(min_length, max_length)

num_new_sequences = len(s_sequences)
new_sequences = []
# intToChar = {1:'A', 2:'T', 3:'C', 4:'G', 5:'N'}
for i in range(num_new_sequences):
    new_sequence = ''
    for j in range(rand_length):
        new_sequence += str(random.randint(1, 5))
    new_sequences.append(new_sequence)

In [429]:
charToInt = {'A': '1', 'T': '2', 'C': '3', 'G': '4', 'N': '5'}
int_s_sequences = []
for seq in s_sequences:
    currIntStr = ''
    for s in seq:
        currIntStr += charToInt[s]
    int_s_sequences.append(currIntStr)

In [430]:
all_sequences = int_s_sequences + new_sequences
all_sequences_labels = [1]  * len(s_sequences) + [0] * len(new_sequences)

In [472]:
X_train, X_test, y_train, y_test = train_test_split(all_sequences, all_sequences_labels, test_size=0.2, random_state=111, shuffle=True)
X_train = [[[int(i) for i in x]] for x in X_train]
X_test = [[[int(i) for i in x]] for x in X_test]
y_train = [[x] for x in y_train]
y_test = [[x] for x in y_test]

# X_test = [[[x]] for x in X_test]
# y_train = [[[x]] for x in y_train]
# y_test = [[[x]] for x in y_test]

# X_test = np.array(X_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)



X_train = tf.ragged.constant(X_train)
# X_test = tf.ragged.constant(X_test)
# y_train = tf.ragged.constant(y_train)
# y_test = tf.ragged.constant(y_test)

In [467]:
print(X_train.shape)
print(y_train.shape)
print(X_train.bounding_shape()[-1])

(384, None, None)
(384, 1)
tf.Tensor(3831, shape=(), dtype=int64)


In [468]:
model = tf.keras.Sequential([
    # Input Layer with shape = [Any,  maximum sequence length]                      
    tf.keras.layers.Input(shape=[None, X_train.bounding_shape()[-1]], batch_size=384, dtype=tf.float32, ragged=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='relu')
])
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
model.summary()

Model: "sequential_81"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_81 (LSTM)               (384, 64)                 997376    
_________________________________________________________________
dense_81 (Dense)             (384, 1)                  65        
Total params: 997,441
Trainable params: 997,441
Non-trainable params: 0
_________________________________________________________________


In [469]:
history = model.fit(X_train, y_train, epochs=10)

Epoch 1/10


InvalidArgumentError:  In[0] mismatch In[1] shape: 3827 vs. 3831: [32,3827] [3831,256] 0 0
	 [[{{node sequential_81/lstm_81/while/body/_1/sequential_81/lstm_81/while/lstm_cell_81/MatMul}}]] [Op:__inference_train_function_98221]

Function call stack:
train_function


# Working Example
#### Below is a working example a ragged tensor input into an LSTM node than outputs a set of numbers
I am unsure of what the differences are and how to improve them

In [522]:
# xx should be 3d for LSTM
xx = tf.ragged.constant(X_train[:320])

"""
Labels represented as OneHotEncoding so you 
should use CategoricalCrossentropy instade of SparseCategoricalCrossentropy
"""

yy = np.array([[1]] * len(X_train[:320]))

In [523]:
print(xx.shape)
print(yy.shape)
print(xx.bounding_shape()[-1])

(320, None, None)
(320, 1)
tf.Tensor(3827, shape=(), dtype=int64)


In [524]:
print(xx[1])

<tf.RaggedTensor [[3, 5, 3, 1, 2, 2, 1, 5, 5, 5, 1, 2, 2, 3, 3, 3, 4, 5, 5, 3, 5, 1, 3, 4, 3, 1, 5, 1, 1, 1, 3, 5, 4, 5, 2, 4, 4, 4, 5, 5, 3, 2, 1, 2, 1, 5, 2, 4, 2, 1, 4, 1, 4, 5, 1, 1, 3, 5, 2, 4, 1, 3, 5, 2, 3, 5, 4, 4, 5, 4, 3, 2, 1, 1, 2, 4, 1, 5, 2, 5, 5, 2, 5, 2, 5, 1, 3, 4, 2, 5, 2, 2, 2, 2, 4, 5, 3, 2, 3, 4, 4, 1, 1, 3, 3, 3, 2, 5, 2, 3, 4, 1, 5, 5, 2, 4, 2, 3, 2, 5, 3, 3, 1, 3, 3, 2, 3, 1, 3, 4, 1, 4, 5, 3, 5, 2, 3, 4, 5, 4, 2, 2, 2, 2, 3, 3, 3, 1, 3, 3, 1, 3, 2, 2, 1, 1, 3, 4, 3, 1, 2, 4, 3, 4, 4, 5, 2, 2, 1, 2, 3, 1, 3, 4, 5, 2, 5, 2, 3, 4, 3, 5, 5, 2, 1, 5, 3, 1, 5, 2, 1, 4, 4, 3, 5, 3, 1, 3, 5, 1, 1, 2, 3, 5, 5, 4, 2, 2, 4, 2, 2, 1, 3, 5, 4, 1, 5, 2, 2, 4, 4, 4, 1, 5, 4, 5, 2, 3, 4, 5, 1, 3, 3, 2, 1, 4, 2, 4, 1, 2, 4, 1, 1, 2, 4, 2, 2, 5, 5, 1, 1, 3, 1, 5, 5, 3, 1, 1, 5, 2, 1, 1, 4, 3, 1, 5, 4, 1, 4, 3, 5, 5, 2, 3, 2, 2, 5, 5, 4, 4, 3, 1, 2, 3, 4, 4, 3, 5, 5, 3, 3, 5, 2, 4, 5, 3, 1, 3, 1, 1, 1, 5, 3, 3, 5, 2, 2, 4, 5, 5, 5, 5, 2, 5, 3, 2, 5, 1, 4, 2, 4, 3, 1, 1, 4, 2, 3, 

In [525]:
# For ragged tensor , get maximum sequence length
max_seq = xx.bounding_shape()[-1]
mdl = tf.keras.Sequential([
    # Input Layer with shape = [Any,  maximum sequence length]                      
    tf.keras.layers.Input(shape=[None, max_seq], batch_size=2, dtype=tf.float32, ragged=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='softmax')
])

# CategoricalCrossentropy
mdl.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

mdl.summary()
history = mdl.fit(xx, yy, epochs=10)


Model: "sequential_94"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_94 (LSTM)               (2, 64)                   996352    
_________________________________________________________________
dense_94 (Dense)             (2, 1)                    65        
Total params: 996,417
Trainable params: 996,417
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
10/10 [==============================] - 2s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/10
10/10 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
10/10 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
10/10 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
10/10 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 1